In [1]:
try:
    from iapws import IAPWS97
except ImportError:
    !pip install iapws
    from iapws import IAPWS97

In [2]:
try:
    from pyfluids import HumidAir, InputHumidAir
except ImportError:
    !pip install pyfluids
    from pyfluids import HumidAir, InputHumidAir

An air-conditioning system operates at a total pressure of 95 kPa and consists of a heating section and a humidifier that supplies wet steam (saturated water vapor) at 100°C. Air enters the heating section at 10°C and 70 percent relative humidity at a rate of 35 m$^3$/min, and it leaves the humidifying section at 20°C and 60 percent relative humidity. 

Determine: 
- (a) the temperature and relative humidity of air when it leaves the heating section,
- (b) the rate of heat transfer in the heating section, and
- (c) the rate at which water is added to the air in the humidifying section.

Answers:
- (a) 19.5 C, 37.7%
- (b) 391 kJ/min
- (c) 0.147 kg/min

In [3]:
# state 1
p1_Pa = 95e3
T1_C = 10 
phi_1_percent = 70
Vdot1_m3PerMin = 35
Vdot1_m3PerS = Vdot1_m3PerMin/60

# state 2
p2_Pa = p1_Pa

# state 3
p3_Pa = p1_Pa
T3_C = 20
phi_3_percent = 60

In [6]:
# define state 1

state1 = HumidAir().with_state(
    InputHumidAir.pressure(p1_Pa),
    InputHumidAir.temperature(T1_C),
    InputHumidAir.relative_humidity(phi_1_percent),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h1_JperKg = state1.enthalpy
h1_kJperKg = h1_JperKg/1e3; print(h1_kJperKg)
w1 = state1.humidity
v1_m3PerKg = state1.specific_volume; print(v1_m3PerKg)

mdotAir_kgPerS = Vdot1_m3PerS/v1_m3PerKg; print(mdotAir_kgPerS)

24.2961764001007
0.8580794146526238
0.6798127578546843


At state 2, we know that we haven't added any water, therefore, $\omega_2 = \omega_1$. We also know that the pressure at $p_2$ is the same as $p_1$. We need one more piece of information in order to have full information at that state. 

In [5]:
w2 = w1 
p2_Pa = p1_Pa

In [9]:
# define state 3
state3 = HumidAir().with_state(
    InputHumidAir.pressure(p3_Pa),
    InputHumidAir.temperature(T3_C),
    InputHumidAir.relative_humidity(phi_3_percent),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

h3_JperKg = state3.enthalpy
h3_kJperKg = h3_JperKg/1e3; print(h3_kJperKg)
w3 = state3.humidity; print(w3)

43.48320052706497
0.009364105201709692


Let's do a mass balance for the water flow:

$$\dot{m}_\text{a}\omega_1 + \dot{m}_\text{w} = \dot{m}_\text{a}\omega_3$$

$$\dot{m}_\text{w} = \dot{m}_\text{a}(\omega_3 - \omega_1)$$

In [12]:
mdotWater_kgPerS = mdotAir_kgPerS*(w3 - w1)
print('(c) Rate of water addition to the humidifying section: {0:1.3f} kg/min'.format(mdotWater_kgPerS*60))

(c) Rate of water addition to the humidifying section: 0.149 kg/min


Let's do an energy balance now between states 2 and 3. 

$$\dot{m}_\text{a} h_2 + \dot{m}_\text{w} h_\text{g} = \dot{m}_\text{a} h_3   $$

$$\implies  h_2 = \dot{m}_\text{a} h_3/\dot{m}_\text{a} - \dot{m}_\text{w} h_\text{g}/\dot{m}_\text{a}  $$

$$\implies  h_2 = h_3 - \dot{m}_\text{w} h_\text{g}/\dot{m}_\text{a}  $$

In [17]:
# !pip install iapws
try:
    from iapws import IAPWS97
except ImportError:
    !pip install iapws
    from iapws import IAPWS97

In [20]:
water = IAPWS97(T=100+273.15,x=1)
hg_kJperKg = water.h; print(hg_kJperKg)

2675.572029220833


In [22]:
h2_kJperKg = h3_kJperKg - (mdotWater_kgPerS/mdotAir_kgPerS)*hg_kJperKg; print(h2_kJperKg)
h2_JperKg = h2_kJperKg*1e3

33.68494093468713


In [24]:
# define state 2

state2 = HumidAir().with_state(
    InputHumidAir.pressure(p2_Pa),
    #InputHumidAir.temperature(T3_C),
    InputHumidAir.humidity(w2),
    InputHumidAir.enthalpy(h2_JperKg),
    #InputHumidAir.relative_humidity(phi_3_percent),
    #InputHumidAir.wet_bulb_temperature(Twb_C),
)

T2_C = state2.temperature; print(T2_C)
phi_2_percent = state2.relative_humidity

19.288313000544292


In [26]:
print('(a) The temperature of air when it leaves the heating section: {0:1.2f} C'.format(T2_C))
print('(a) The humidity of air when it leaves the heating section: {0:1.2f} %'.format(phi_2_percent))

(a) The temperature of air when it leaves the heating section: 19.29 C
(a) The humidity of air when it leaves the heating section: 38.41 %


The rate of heat transfer is then:
$$\dot{Q}_\text{in} = \dot{m}_\text{a} (h_2 - h_1) $$


In [28]:
QdotIn_kJperS = mdotAir_kgPerS*(h2_kJperKg - h1_kJperKg)

In [30]:
print('(b) The rate of heat transfer in the heating section: {0:1.2f} kJ/min'.format(QdotIn_kJperS*60))

(b) The rate of heat transfer in the heating section: 382.96 kJ/min


#### Checking water property values with those of CoolProp

Let's determine the enthalpy of the water vapor using CoolProp.

In [31]:
!pip install CoolProp

In [32]:
from CoolProp.CoolProp import PropsSI

In [35]:
hg_JperKg_ = PropsSI('H','T',100+273.15,'Q',1,'water')
hg_kJperKg_ = hg_JperKg_/1000; print(hg_kJperKg_)

error = (hg_kJperKg - hg_kJperKg_)/hg_kJperKg; print(error)

2675.5698844194585
8.01623485014104e-07
